In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import pickle

In [5]:
# Load the all trained model
model = load_model('model.h5')

# Load the encoder and scaler
with open('onehot_encoder_geo.pickle', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pickle', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pickle', 'rb') as file:
    scaler = pickle.load(file)



In [4]:
# Example new data point

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [8]:
# Geography encoding

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/opt/anaconda3/envs/venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [13]:
# Encode label encoder - gender column

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [14]:
# combime one hot encoded columns with original dataframe


input_df = pd.concat([input_df,geo_encoded_df],axis=1)
input_df = input_df.drop(['Geography'],axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
# Scaling the input data

input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [17]:
# Prediction

pred = model.predict(input_scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


array([[0.0557045]], dtype=float32)

In [19]:
pred > 0.5

array([[False]])

In [18]:
prediction = (pred > 0.5).astype(int)
if prediction[0][0] == 1:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.
